In [8]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [2]:
#重建索引
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=["d", "b", "a", "c"])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

In [3]:
obj2 = obj.reindex(["a", "b", "c", "d", "e"])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

对该Series调用reindex将会根据新索引进行重新排序。
如果某个值不存在则会出现缺失值
对于缺失值，一般我们要进行插填或填值处理。
method选项可以达到此目的，比如使用ffill可以实现向前充值：

In [5]:
obj3 = pd.Series(["bule", "purple", "yellow"], index=[0, 2, 4])
obj3

0      bule
2    purple
4    yellow
dtype: object

In [6]:
obj3.reindex(np.arange(6), method="ffill")

0      bule
1      bule
2    purple
3    purple
4    yellow
5    yellow
dtype: object

借助DataFrame, reindex可以修改（行）索引、列，也可以同时修改。
只传入一个序列时，会重建索引中的行：

In [10]:
frame = pd.DataFrame(np.arange(9).reshape((3, 3)),
                     index=["a", "c", "d"],
                     columns=["Ohio", "Texas", "California"])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [11]:
frame2 = frame.reindex(index=["a", "b", "c", "d"])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [12]:
#用columns关键字重建索引：
states = ["Texas", "Utah", "California"]
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


"Ohio"不在states中，所以这一列被删除。
另一种重建索引的方法是传入新的轴标签作为位置参数，然后用axis关键字对指定轴进行重建索引：

In [13]:
frame.reindex(states, axis="columns")

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


frame.reindex(states, axis="columns")
用于重新排列或过滤 DataFrame 的列（或行），以匹配指定的索引或列名顺序。
这个方法可以用于 DataFrame 的列（axis="columns")或行（axis="index"）。

重新排列列或行：根据指定顺序重新排列 DataFrame 中的列或行。
添加缺失的列或行：如果指定的列或行在原始 DataFrame 中不存在，会被添加且值为缺失值（NaN）。
保持 DataFrame 的一致性：当你需要多个 DataFrame 有相同的列或行顺序时，使用 reindex 很有用。

![jupyter](./5.4.png)

In [15]:
#还可以用loc运算符重建索引，这是最常用的方法，只有在新索引DF中已经存在时，才能这么做
#否则 reindex 会给新标签插入缺失值
frame.loc[["a", "d", "c"], ["California", "Texas"]]

,California,Texas
a,2,1
d,8,7
c,5,4


In [ ]:
frame.loc[["a", "d", "c", "e"], ["California", "Texas"]]
这种写法是错误的，因为loc不能直接创建索引，所以有多的索引进来后会报错TypeError

In [17]:
#删除指定轴上的项
obj = pd.Series(np.arange(5.), index=["a", "b", "c", "d", "e"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [18]:
new_obj = obj.drop("c")
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [19]:
obj.drop(["d", "c"])

a    0.0
b    1.0
e    4.0
dtype: float64

删除某条轴上的一个或多个项只要有一个索引数组或不包含这些项的列表，就可以使用reindex方法或基于.loc的索引进行删除。
由于需要执行一些数据整理和集合逻辑操作，因此drop方法返回的是一个在指定轴上删除了指定值的新对象。

对于DF，可以删除任意轴上的索引值，先创建一个DF实例

In [21]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)),
                    index=["Ohio", "Colorado", "Utah", "New York"],
                    columns=["one", "two", "three", "four"])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [22]:
#用标签序列调用drop会从行标签（axis=0）删除值
data.drop(index=["Colorado", "Ohio"])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [ ]:
这里是drop(index=["Colorado", "Ohio"])，传入指定行 index 进行删除
drop 默认从行标签中删除指定的标签（axis=0）行删除。
可以通过设置 axis=1 从列标签中删除指定的标签。
使用 inplace=True 可以直接修改原 DataFrame，而不是创建新的 DataFrame

In [23]:
#通过传入指定列 columns 删除指定列标签
data.drop(columns=["two"])

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [24]:
#可以传入 axis=0 或 axis=1 进行行、列删除
data.drop("two", axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [25]:
data.drop(["two", "four"], axis="columns")

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


In [26]:
#索引、选取和过滤
#Series索引(obj[...])的工作方式类似于Numpy数组的索引，只不过Series的索引值可以不仅仅是整数。
obj = pd.Series(np.arange(4.), index=["a", "b", "c", "d"])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [27]:
obj["b"]

1.0

In [28]:
obj[1]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_14913/2469632899.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[1]


1.0

这个警告是 Pandas 将来会改变 Series.__getitem__ 的行为。
当使用整数索引来访问 Series 对象时，当前版本的 Pandas 允许你通过位置（position）访问值，
但在未来的版本中，这种方式将被弃用，并且整数将总是被视为标签（labels）。

In [30]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [31]:
obj[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

In [32]:
obj[[1, 3]]

/var/folders/_c/cfwdc6fn7r59wgd23s5wyx580000gn/T/ipykernel_14913/2982346117.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  obj[[1, 3]]


b    1.0
d    3.0
dtype: float64

In [33]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

即，在DF中搜索时，可以不仅仅只搜索index，也可以是后面的数值，上述写法可以用来选取数据，
但有更可靠的方法，利用loc运算符选取索引值：

In [34]:
obj.loc[["b", "a", "d"]]

b    1.0
a    0.0
d    3.0
dtype: float64

In [ ]:
loc方式更可取，因为用[]进行索引时，针对整数的处理有所不同。
如果索引包含整数，常规的基于[]的索引会将整数用作标签，因此索引的选取取决于数据类型。
如下：

In [ ]:
obj1 = pd.Series([1, 2, 3], index=[2, 0, 1])
obj2 = pd.Series([1, 2, 3], index=[""a])